In [1]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV
import time

In [13]:
train = pd.read_csv('trainC.csv', keep_default_na=False)
train = pd.get_dummies(train)
features = train.iloc[:, train.columns != 'SalePrice']
labels = train['SalePrice']

In [14]:
train.isnull().sum().sort_values(ascending= False).head(5)

SaleCondition_Partial    0
BsmtFinSF2_219           0
BsmtFinSF2_273           0
BsmtFinSF2_264           0
BsmtFinSF2_258           0
dtype: int64

In [15]:
start = time.time()
reg = LinearRegression()
reg.fit(features, labels)
end = time.time()

print end - start

1.5490000248


In [17]:
pred = reg.predict(features)
temp = pd.DataFrame(pred - labels)
print temp.describe()
print reg.score(features, labels)

          SalePrice
count  1.460000e+03
mean  -1.147010e-09
std    5.746545e-07
min   -1.556706e-06
25%   -3.252644e-07
50%   -2.101297e-08
75%    2.807647e-07
max    7.715076e-06
1.0
